In [1]:
import torch
# ^^^ pyforest auto-imports - don't write above this line
import pandas as pd, numpy as np
from glob import glob
from tqdm.notebook import tqdm

In [7]:
path.split("/")[-1]

'p000001.psv'

In [8]:
dfA = []
for path in tqdm(glob('/training_setA/*.psv')):
    df = pd.read_csv(path, sep="|")
    pid = int(path.split("/")[-1][1:-4])
    df['pid']=pid
    dfA.append(df)

dfB=[]
for path in tqdm(glob('/training_setB/*.psv')):
    df = pd.read_csv(path, sep="|")
    pid = int(path.split("/")[-1][1:-4])
    df['pid'] = pid
    dfB.append(df)  

  0%|          | 0/20336 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

In [9]:
dfA = pd.concat(dfA)
dfB = pd.concat(dfB)

In [10]:
df = pd.concat([dfA,dfB]).reset_index(drop=True)

In [16]:
len(dfA), len(dfB)

(790215, 761995)

### train, test 분리

In [51]:
# 3만명의 샘플을 가져온다.
train_pids = df.pid.sample(30000, replace=False, random_state=1234)
train_df = df.query('pid in @train_pids').reset_index(drop=True)
test_df = df.query('pid not in @train_pids').reset_index(drop=True)

### Nan 값 채우기 (median & forward fill)

In [52]:
vital_cols=['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
       'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
       'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
       'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets']

col_stat = {}

for col in tqdm(vital_cols):
    col_stat[col] = {}
    col_stat[col]['mean']=train_df[col].median()
    col_stat[col]['median'] = train_df[col].median()
    col_stat[col]['std'] = train_df[col].std()
    
    first_index = train_df.drop_duplicates(['pid'], keep='first').index
    train_df.loc[first_index, col] = train_df.loc[first_index,col].apply(lambda x:col_stat[col]['median'] if pd.isna(x) else x)
    train_df[col] = train_df[col].fillna(method='ffill')
    
    first_index = test_df.drop_duplicates(['pid'], keep='first').index
    test_df.loc[first_index, col] = test_df.loc[first_index,col].apply(lambda x:col_stat[col]['median'] if pd.isna(x) else x)
    test_df[col] = test_df[col].fillna(method='ffill')

  0%|          | 0/34 [00:00<?, ?it/s]

In [ ]:
NEWS score ['HR']

In [27]:
def get_SOFA(sofa_input):
    # TODO: SOFA 계산 구현
    
    return sofa_score

test_df['SOFA'] = 0
sofa_columns = [] #SOFA에서 쓰는 컬럼 리스트
for i in tqdm(test_df.index):
    sofa_input = test_df.loc[i, sofa_columns]
    test_df.at[i, 'SOFA'] = get_SOFA(sofa_input)

Index(['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
       'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
       'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
       'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2',
       'HospAdmTime', 'ICULOS', 'SepsisLabel', 'pid'],
      dtype='object')

In [29]:
sofa_columns = ['BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2']

In [32]:
sofa_input = test_df.loc[123, sofa_columns]

In [33]:
sofa_input

BaseExcess     0.00
HCO3          28.00
FiO2           0.50
pH             7.39
PaCO2         40.00
Name: 123, dtype: float64

In [47]:
train_df

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,pid
0,84.0,98.0,37.00,121.0,80.67,62.0,18.0,33.0,0.0,24.0,...,253.0,180.0,45.82,0,1.0,0.0,-1195.71,1,0,3
1,87.0,96.0,37.00,131.0,70.33,62.0,29.0,33.0,0.0,24.0,...,253.0,180.0,45.82,0,1.0,0.0,-1195.71,2,0,3
2,93.0,97.0,37.11,130.0,69.33,62.0,40.0,33.0,0.0,24.0,...,253.0,180.0,45.82,0,1.0,0.0,-1195.71,3,0,3
3,90.0,95.0,37.11,128.0,69.33,62.0,23.0,33.0,0.0,24.0,...,253.0,180.0,45.82,0,1.0,0.0,-1195.71,4,0,3
4,89.0,97.0,37.11,137.0,62.67,50.0,26.0,33.0,0.0,24.0,...,253.0,180.0,45.82,0,1.0,0.0,-1195.71,5,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882439,76.0,97.0,36.50,132.0,88.00,61.0,23.0,33.0,0.0,24.0,...,253.0,180.0,45.00,0,NaN,NaN,-106.79,18,0,113144
882440,76.0,97.0,36.50,132.0,88.00,61.0,23.0,33.0,0.0,24.0,...,253.0,180.0,45.00,0,NaN,NaN,-106.79,19,0,113144
882441,66.0,95.0,36.50,128.0,94.00,71.0,18.0,33.0,0.0,24.0,...,253.0,180.0,45.00,0,NaN,NaN,-106.79,20,0,113144
882442,66.0,95.0,36.50,128.0,94.00,71.0,18.0,33.0,0.0,24.0,...,253.0,180.0,45.00,0,NaN,NaN,-106.79,21,0,113144


### Normalize

In [53]:
for col in tqdm(vital_cols) :
    train_df[col] = (train_df[col]-col_stat[col]['mean'])/col_stat[col]['std']

  0%|          | 0/34 [00:00<?, ?it/s]

In [54]:
for col in tqdm(vital_cols) :
    test_df[col] = (test_df[col]-col_stat[col]['mean'])/col_stat[col]['std']

  0%|          | 0/34 [00:00<?, ?it/s]

In [131]:
train_df

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,...,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,pid
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,45.82,0,1.0,0.0,-1195.71,1,0,3
1,0.171839,-0.684893,0.000000,0.425936,-0.632096,0.000000,2.128146,0.0,0.0,0.0,...,0.0,0.0,45.82,0,1.0,0.0,-1195.71,2,0,3
2,0.515517,-0.342446,0.142037,0.383342,-0.693228,0.000000,4.256291,0.0,0.0,0.0,...,0.0,0.0,45.82,0,1.0,0.0,-1195.71,3,0,3
3,0.343678,-1.027339,0.142037,0.298155,-0.693228,0.000000,0.967339,0.0,0.0,0.0,...,0.0,0.0,45.82,0,1.0,0.0,-1195.71,4,0,3
4,0.286398,-0.342446,0.142037,0.681497,-1.100361,-0.861514,1.547742,0.0,0.0,0.0,...,0.0,0.0,45.82,0,1.0,0.0,-1195.71,5,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882439,-0.458238,-0.342446,-0.645623,0.468529,0.448092,-0.071793,0.967339,0.0,0.0,0.0,...,0.0,0.0,45.00,0,NaN,NaN,-106.79,18,0,113144
882440,-0.458238,-0.342446,-0.645623,0.468529,0.448092,-0.071793,0.967339,0.0,0.0,0.0,...,0.0,0.0,45.00,0,NaN,NaN,-106.79,19,0,113144
882441,-1.031035,-1.027339,-0.645623,0.298155,0.814879,0.646136,0.000000,0.0,0.0,0.0,...,0.0,0.0,45.00,0,NaN,NaN,-106.79,20,0,113144
882442,-1.031035,-1.027339,-0.645623,0.298155,0.814879,0.646136,0.000000,0.0,0.0,0.0,...,0.0,0.0,45.00,0,NaN,NaN,-106.79,21,0,113144


In [ ]:
'''
할일 : train_df에서 batch_size만큼 length=30 seq 데이터를 추출해와서 배치 만들어서 모델에 집어넣기
+ 패혈증, 일반 데이터 균등 샘플링
'''

In [57]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, WeightedRandomSampler

In [108]:
def _make_window(rows, group, feature_columns, sequence_length):
        X = []
        for i in range(len(rows)):
            row = rows.iloc[i]

            df = group.get_group((row.pid))
            df = df.loc[:row.name]
            features = df[feature_columns].values

            if len(features) < sequence_length:
                x = np.zeros([sequence_length, len(feature_columns)])
                x[-len(features):] = features
            else:
                x = features[-sequence_length:]

            X.append(x)

        return X
    
class Physionet2019(Dataset):
    def __init__(self, df):
        self.group = df.groupby(['pid'])
        self.seq_len = 30
        self.df = df 
        self.feat_cols = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
                           'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
                           'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
                           'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
                           'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
                           'Fibrinogen', 'Platelets']
        self.info_cols = ['pid', 'HospAdmTime', 'ICULOS', 'SepsisLabel', 'Age','Gender']

    def __len__(self):
        return len(self.df)

    def get_x_y(self, idx, df):
        rows = df.iloc[[idx]]
        x = _make_window(rows, self.group, self.feat_cols, self.seq_len)
        y = list(rows['SepsisLabel'].values)
        return x, y, rows
    
    def __getitem__(self, idx):
        x, y, rows = self.get_x_y(idx, self.df)
        return x, y, list(rows[self.info_cols].values)

In [132]:
train_ds = Physionet2019(train_df)
test_ds = Physionet2019(test_df)

In [134]:
x, y, info = test_ds[10]

In [129]:
def get_sampler(train_df):
    class_counts = train_df['SepsisLabel'].value_counts() 
    num_samples = sum(class_counts)
    labels = train_df['SepsisLabel'].to_list()

    #클래스별 가중치 부여 [48000/43200, 48000/4800] => class 1에 가중치 높게 부여하게 됨
    class_weights = [num_samples / class_counts[i] for i in range(len(class_counts))] 

    # 해당 데이터의 label에 해당되는 가중치
    weights = [class_weights[labels[i]] for i in range(int(num_samples))] #해당 레이블마다의 가중치 비율
    sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))
    return sampler

In [ ]:
def val_collate(batch):
    X = []
    Label = []
    INFO = []
    for x, label, info in batch:
        for i in range(len(x)):
            X.append(x[i])
            Label.append(label[i])
            INFO.append(info[i])

    X_ = [torch.Tensor(x) for x in X]
    xs = torch.stack(X_, dim=0)
    return xs, torch.Tensor(Label), np.array(INFO)

In [142]:
train_dl = DataLoader(train_ds, collate_fn=val_collate, sampler=get_sampler(train_df), batch_size=128)

<IPython.core.display.Javascript object>

In [143]:
# 학습
for epoch in range(10):
    for data in train_dl:
        x, y, info = data
        pred = model(x)
        loss = loss_fn(y, pred)
        optii~~
        ~~
#평가
test_Y, test_pred, test_info = [], [], []
for data in test_dl:
    x, y, info = data
    pred = model(x)
    test_pred.append(pred)
    test_info.append(info)
    
test_info = np.concatenate(test_info)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [148]:
Y = np.concatenate(Y)

In [150]:
len(train_dl)

6895